In [ ]:
import pandas as pd
import os


INPUT_FOLDER = "filtered_data"
RESULTS_FOLDER = "results"
os.makedirs(RESULTS_FOLDER, exist_ok=True)


selected_types = ["ліцей", "гімназія", "навчально-виховний комплекс", 
                  "спеціалізована школа", "заклад фахової передвищої освіти", "середня загальноосвітня школа"]

def count_education_types_by_location(df, year):
    required_cols = ['education_org_type', 'territory_type']
    if not all(col in df.columns for col in required_cols):
        print(f"⚠️ Відсутні колонки: education_org_type або territory_type у файлі для {year}")
        return None
    
    df_filtered = df[df['education_org_type'].isin(selected_types)].copy()
    
    valid_locations = df_filtered['territory_type'].dropna().unique().tolist()
    if not valid_locations:
        print(f"⚠️ Жодних даних про territory_type для {year}")
        return None

    result = df_filtered.groupby(['territory_type', 'education_org_type']).size().unstack(fill_value=0)

    result = result.loc[valid_locations]
    
    return result


all_results = {}
for filename in os.listdir(INPUT_FOLDER):
    if not filename.endswith(".csv"):
        continue
    
    file_path = os.path.join(INPUT_FOLDER, filename)
    year = filename.split('.')[0]  
    print(f"🔄 Обробка {filename} ({year})...")
    
    try:
        df = pd.read_csv(file_path, encoding='utf-8', on_bad_lines='skip', low_memory=False)
        print(f"   Завантажено {len(df)} рядків, {len(df.columns)} стовпців")
        
        result = count_education_types_by_location(df, year)
        if result is not None:
            all_results[year] = result
            print(f"✅ Дані для {filename} ({year}) оброблено")
    
    except Exception as e:
        print(f"❌ Помилка у файлі {filename}: {e}")
        import traceback
        traceback.print_exc()


if all_results:
    for year, result in all_results.items():
        output_path = os.path.join(RESULTS_FOLDER, f"education_types_by_location_{year}.csv")
        result.to_csv(output_path)
        print(f"✅ Результати для {year} збережено у: {output_path}")
else:
    print("⚠️ Жодних даних не оброблено")

print(f"\n🎉 Аналіз завершено! Результати збережено в '{RESULTS_FOLDER}'")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os


INPUT_FOLDER = "filtered_data"
RESULTS_FOLDER = "results"
GRAPHS_FOLDER = os.path.join(RESULTS_FOLDER, "graphs")
for folder in [RESULTS_FOLDER, GRAPHS_FOLDER]:
    os.makedirs(folder, exist_ok=True)

selected_types = ["ліцей", "гімназія", "навчально-виховний комплекс", 
                  "спеціалізована школа", "заклад фахової передвищої освіти", "середня загальноосвітня школа"]

def visualize_education_types_by_location_2022():
    filename = "2022.csv"
    file_path = os.path.join(INPUT_FOLDER, filename)
    
    print(f"🔄 Обробка {filename}...")
    
    try:
        df = pd.read_csv(file_path, encoding='utf-8', on_bad_lines='skip', low_memory=False)
        print(f"   Завантажено {len(df)} рядків, {len(df.columns)} стовпців")
        
        required_cols = ['education_org_type', 'territory_type']
        if not all(col in df.columns for col in required_cols):
            print("⚠️ Відсутні колонки: education_org_type або territory_type")
            return
        
        df_filtered = df[df['education_org_type'].isin(selected_types)].copy()
        
        locations = ["селище, село", "селище міського типу", "місто"]
        df_filtered = df_filtered[df_filtered['territory_type'].isin(locations)]
        
        if df_filtered.empty:
            print("⚠️ Жодних даних про territory_type серед заданих категорій")
            return
        
        total_locations = df_filtered['territory_type'].value_counts().reindex(locations, fill_value=0)
        
        result = df_filtered.groupby(['territory_type', 'education_org_type']).size().unstack(fill_value=0)
        
        for edu_type in selected_types:
            if edu_type not in result.columns:
                result[edu_type] = 0
        
        result = result.reindex(index=locations)
        
        print(f"   Наявні типи закладів: {list(result.columns)}")
        
        relative_result = result.div(total_locations, axis=0)
        
        relative_result_long = relative_result.reset_index().melt(
            id_vars='territory_type',
            var_name='education_org_type',
            value_name='relative_count'
        )
        
        sns.set_style("whitegrid")
        plt.figure(figsize=(10, 6))

        g = sns.catplot(
            x='territory_type',
            y='relative_count',
            hue='education_org_type',
            data=relative_result_long,
            kind='bar',
            height=6,
            aspect=1.5,
            palette='Set3',
            alpha=0.8
        )
        
        g.fig.suptitle("Relative distribution of types of establishments by type of settlement (2022)", y=1.05)
        g.set_axis_labels("Type of settlement", "Relative number of establishments per settlement")
        g.set_xticklabels(rotation=45, ha='right')

        g._legend.set_bbox_to_anchor((1.05, 1.1)) 
        g._legend.set_loc('upper right') 
        
        graph_path = os.path.join(GRAPHS_FOLDER, "education_types_by_location_2022_relative.png")
        g.figure.tight_layout()
        g.figure.savefig(graph_path, dpi=300, bbox_inches="tight")
        plt.close()
        print(f"✅ Графік збережено у: {graph_path}")

    
    except Exception as e:
        print(f"❌ Помилка у файлі {filename}: {e}")
        import traceback
        traceback.print_exc()

visualize_education_types_by_location_2022()

print(f"\n🎉 Аналіз завершено! Результати збережено в '{RESULTS_FOLDER}'")

In [31]:
import pandas as pd
import os
import re

INPUT_FOLDER = "filtered_data"
RESULTS_FOLDER = "results"
for folder in [RESULTS_FOLDER]:
    os.makedirs(folder, exist_ok=True)

selected_types = ["ліцей", "гімназія", "навчально-виховний комплекс", 
                  "спеціалізована школа", "заклад фахової передвищої освіти", "середня загальноосвітня школа"]

def analyze_averages_by_year():
    for filename in os.listdir(INPUT_FOLDER):
        if not filename.endswith(".csv"):
            continue
        year = re.search(r'(\d{4})', filename)
        if not year:
            print(f"⚠️ Неможливо визначити рік для {filename}")
            continue
        year = year.group(1)
        
        file_path = os.path.join(INPUT_FOLDER, filename)
        print(f"🔄 Обробка {filename} ({year})...")
        
        try:
            df = pd.read_csv(file_path, encoding='utf-8', on_bad_lines='skip', low_memory=False)
            print(f"   Завантажено {len(df)} рядків, {len(df.columns)} стовпців")

            required_cols = ['education_org_type', 'territory_type', 'average_score']
            if not all(col in df.columns for col in required_cols):
                print(f"⚠️ Відсутні колонки: {', '.join(col for col in required_cols if col not in df.columns)} у {filename}")
                continue

            df_filtered = df[df['education_org_type'].isin(selected_types)].copy()

            locations = ["селище, село", "селище міського типу", "місто", "інша країна", "село"]
            df_filtered = df_filtered[df_filtered['territory_type'].isin(locations)]
            
            if df_filtered.empty:
                print(f"⚠️ Жодних даних для вибраних типів закладів або поселень у {filename}")
                continue
                
            avg_result = df_filtered.groupby(['territory_type', 'education_org_type']).agg({
                'average_score': 'mean', 
                'average_score': 'size'   
            }).reset_index()
            avg_result.rename(columns={'average_score': 'count'}, inplace=True)  
            avg_result['average_score'] = df_filtered.groupby(['territory_type', 'education_org_type'])['average_score'].mean().values 

            avg_result['year'] = year

            output_path = os.path.join(RESULTS_FOLDER, f"average_scores_{year}.csv")
            avg_result.to_csv(output_path, index=False)
            print(f"✅ Табличка для {year} збережено у: {output_path}")
        
        except Exception as e:
            print(f"❌ Помилка у файлі {filename}: {e}")
            import traceback
            traceback.print_exc()

    print(f"\n🎉 Аналіз завершено! Результати збережено в '{RESULTS_FOLDER}'")

analyze_averages_by_year()

🔄 Обробка 2020.csv (2020)...
   Завантажено 201212 рядків, 131 стовпців
✅ Табличка для 2020 збережено у: results/average_scores_2020.csv
🔄 Обробка 2021.csv (2021)...
   Завантажено 188609 рядків, 152 стовпців
✅ Табличка для 2021 збережено у: results/average_scores_2021.csv
🔄 Обробка 2023.csv (2023)...
   Завантажено 256313 рядків, 64 стовпців
✅ Табличка для 2023 збережено у: results/average_scores_2023.csv
🔄 Обробка 2022.csv (2022)...
   Завантажено 213647 рядків, 36 стовпців
✅ Табличка для 2022 збережено у: results/average_scores_2022.csv
🔄 Обробка 2019.csv (2019)...
   Завантажено 172734 рядків, 131 стовпців
✅ Табличка для 2019 збережено у: results/average_scores_2019.csv
🔄 Обробка 2024.csv (2024)...
   Завантажено 264164 рядків, 78 стовпців
✅ Табличка для 2024 збережено у: results/average_scores_2024.csv

🎉 Аналіз завершено! Результати збережено в 'results'
